# Maximum Channel Capacity of a multi-threaded program 

In [1]:
import os
import re
import numpy as np
import math
import copy
import pandas as pd

# Input File

In [2]:
########################################inputFile#######################################################
def func_input_file (pathString):
    f = open (pathString, "r")
    link_file = f.read () 
    print(pathString)
    return link_file

####################################splitInputString####################################################
def func_split_input_string (tempString):
    l_tmp = tempString.split("\n")
    del l_tmp[-1] #delete the additional '' in the end of l_tmp
    
    num_secrets = int(l_tmp [0])
#     print("num_secrets")
#     print(num_secrets)
    del l_tmp[0] #delete the row of num_secrets from l_temp
    
    num_traces = len(l_tmp)
#     print("\nnum_traces")
#     print(num_traces)
    
    l_prob_trace = [[0 for i in range (num_secrets)] for i in range (num_traces)]  
    l_freq_trace = [[0 for i in range (num_secrets)] for i in range (num_traces)]  
    l_name_trace = []

    for idx_trace in range(num_traces):
        tmp = l_tmp[idx_trace]
        tmp = tmp.split(":")
        #print(tmp)
        
        l_name_trace.append(tmp[0])
        del tmp[0]
        
        for i in "{ } ":#removing '{', '}',and space
            tmp[0] = tmp[0].replace(i, '')#freq
            tmp[1] = tmp[1].replace(i, '')#prob
        #print(tmp)
        
        tmp[0] = re.split('=|,', tmp[0])#spliting based on '=' or ','
        tmp[1] = re.split('=|,', tmp[1])
        
        #because of starting at 1, I reduces the indeces by 1
        for j in range(0,len(tmp[0]),2):
            l_freq_trace[idx_trace][int(tmp[0][j])-1] = int(tmp[0][j+1])
        for j in range(0,len(tmp[1]),2):
            l_prob_trace[idx_trace][int(tmp[1][j])-1] = float(tmp[1][j+1])
        
#     print("\nl_name_trace")
#     print(l_name_trace)   
    
#     print("\nfreq_trace")
#     print(l_freq_trace)

#     print("\nprob_trace")
#     print(l_prob_trace)
    
    
    return num_secrets , num_traces, l_prob_trace, l_name_trace, l_freq_trace

####################################printInputFile####################################################
def func_print_input_file(num_secrets, num_traces, l_name_trace, l_freq_trace, l_prob_trace):
    print("num_secrets")
    print(num_secrets)

    print("\nnum_traces")
    print(num_traces)

    print("\nl_name_trace")
    print(l_name_trace)   
    
    print("\nfreq_trace")
    print(l_freq_trace)

    print("\nprob_trace")
    print(l_prob_trace)

# Store in file

In [3]:
def func_store_excel(str_input, l_result):
    str_input = str_input.replace('/', '_')
    writer = pd.ExcelWriter(str_input + '.xlsx')
    df_output = pd.DataFrame(l_result, columns = ['fitness', 'sum_prob_chromosome', 'chromosome'])
    df_output.to_excel(writer, str_input)
    writer.save()

# Class GA

In [4]:
class GA:
    """
    Implementing the Genetic Algorithms
    whithout sorting and natural selection
    based on binary representation
    """
    def __init__(self, num_pop, num_secrets, num_bit, num_traces, l_prob_trace, l_name_trace, l_freq_trace,\
                 val_mutation_prob, val_mating_prob, val_max_iter):
        self.num_pop = num_pop
        self.num_secrets = num_secrets
        self.num_bit = num_bit
        self.num_traces = num_traces
        self.l_prob_trace = l_prob_trace
        self.l_name_trace = l_name_trace
        self.l_freq_trace = l_freq_trace
        self.val_mating_prob = val_mating_prob
        self.val_mutation_prob = val_mutation_prob
        self.val_max_iter = val_max_iter
        self.l_result = []
    
    def m_create_pop(self):
        """
        create the 1st population of size "num_pop * num_secrets * num_bit"
        """
        self.l_pop = [] #list of population
        for i in range (self.num_pop):
            self.l_pop.append(np.random.randint(2,size = (self.num_secrets, self.num_bit)))
        
     
    def m_bin_2_float(self, listEval):
        """
        convert binary to float of list of 1s and 0s of size num_bit
        """
        val_temp = 0
        p = 1
        for i in range(self.num_bit):
            p = p/2
            val_temp = val_temp + listEval[i] * p

        return val_temp
    
    def m_fitness_value (self, l_val_pop):
        """
        compute the fitness values of all chromosomes in the population
        based on min-entropy
        """
        l_fitness_value = [] #fitness value of all chromosomes in the l_val_pop

        for idx_pop in range(self.num_pop):
            l_val_h = l_val_pop[idx_pop]
            val_initial_uncer = - math.log2(max(l_val_h))
            l_trace_final_uncer = []

            for idx_trace in range(self.num_traces):
                val_sum_mu_freq = sum(l_val_h[i] * self.l_freq_trace[idx_trace][i] for i in range(self.num_secrets))

                if(val_sum_mu_freq != 0):
                    l_mu_h_s_f_T = list(l_val_h[i] * self.l_freq_trace[idx_trace][i] / val_sum_mu_freq\
                                      for i in range(self.num_secrets))
                else:
                    l_mu_h_s_f_T = list(0 for i in range(self.num_secrets))


                if (max(l_mu_h_s_f_T) != 0):
                    l_trace_final_uncer.append(- math.log2(max(l_mu_h_s_f_T)))
                else:
                    l_trace_final_uncer.append(0)

            val_final_uncer = min(l_trace_final_uncer[i] for i in range(self.num_traces))

            val_sum_h_penalty = sum(l_val_h)
            l_fitness_value.append(val_initial_uncer - val_final_uncer - abs(val_sum_h_penalty - 1) * 7)
        return l_fitness_value
    
    def m_mating(self, npArray_parent_1, npArray_parent_2):
        """
        applying the single-point mating of probability self.val_mating_prob
        """
        if np.random.uniform() < self.val_mating_prob:
            idx_mating_point = np.random.randint(0, self.num_secrets * self.num_bit)

            npArray_offspring_1 = np.append(npArray_parent_1.ravel() [:idx_mating_point],\
                                            npArray_parent_2.ravel() [idx_mating_point:])
            npArray_offspring_2 = np.append(npArray_parent_2.ravel() [:idx_mating_point],\
                                            npArray_parent_1.ravel() [idx_mating_point:])

            npArray_offspring_1 = npArray_offspring_1.reshape((self.num_secrets, self.num_bit))
            npArray_offspring_2 = npArray_offspring_2.reshape((self.num_secrets, self.num_bit))
            return npArray_offspring_1, npArray_offspring_2

        return npArray_parent_1, npArray_parent_2
        
    def m_mutation(self, npArray_offspring):
        """
        applying the classical mutation of probability self.val_mutation_prob
        """
        if np.random.uniform() < self.val_mutation_prob:
            idx_mutation_point = np.random.randint(0, self.num_secrets * self.num_bit)
            npArray_offspring_mutated = copy.deepcopy(npArray_offspring)
            npArray_offspring_mutated.ravel()[idx_mutation_point] = 1 - npArray_offspring_mutated.ravel()[idx_mutation_point]
            return npArray_offspring_mutated
        return npArray_offspring
    
    def m_start(self, max_num_times):
        """
        Starting
        """
        for num_times in range(max_num_times):

            self.m_create_pop()

            for num_iteration in range(1, self.val_max_iter + 1):
                
                l_val_pop = []
                
                for idx_pop in range(self.num_pop):
                    l_val_pop.append(list(self.m_bin_2_float(self.l_pop[idx_pop][i])\
                                          for i in range(self.num_secrets)))

                l_fitness_value = self.m_fitness_value (l_val_pop)
#                 print(l_fitness_value)

                val_fitness_elite = np.max(l_fitness_value)
                idx_fitness_elite = np.argmax(l_fitness_value)
                l_val_best_chromosome = list(self.m_bin_2_float(self.l_pop[idx_fitness_elite][i])\
                                         for i in range(self.num_secrets))
                
                val_fitness_worst = np.min(l_fitness_value)

                l_new_pop = []
                l_new_pop.append(self.l_pop[idx_fitness_elite])
#                 print(self.l_pop[idx_fitness_elite])
#                 print(l_new_pop)
#                 print(val_fitness_elite)

                for i in range(self.num_pop):
                    l_fitness_value[i] = (l_fitness_value[i] - val_fitness_worst \
                                               + abs(val_fitness_worst/10))
                
                val_sum_prob = sum(l_fitness_value)
                l_prob_fitness = list(l_fitness_value[i] / val_sum_prob for i in range(self.num_pop))
                
                for i in range(1, self.num_pop - 1, 2):
                    idx_parent_1, idx_parent_2 = np.random.choice(list(range(self.num_pop)), size=2,\
                                                                  p=l_prob_fitness, replace=True)
                    
                    npArray_offspring_1, npArray_offspring_2 = self.m_mating(self.l_pop[idx_parent_1],\
                                                                             self.l_pop[idx_parent_2])
                    
                    l_new_pop.append(self.m_mutation(npArray_offspring_1))
                    l_new_pop.append(self.m_mutation(npArray_offspring_2))


                idx_parent_1, idx_parent_2 = np.random.choice(list(range(self.num_pop)), size=2,\
                                                              p=l_prob_fitness, replace=True)
                
                npArray_offspring_1, npArray_offspring_2 = self.m_mating(self.l_pop[idx_parent_1],\
                                                                         self.l_pop[idx_parent_2])

                l_new_pop.append(self.m_mutation(npArray_offspring_1))
                
#                 print("Round: {0}\tIter: {1}\tElite fitness: {2}".format(num_times, num_iteration, val_fitness_elite))
                
                self.l_pop = []
                self.l_pop = copy.deepcopy(l_new_pop)
#                 for i in range(self.num_pop):
#                     print(self.l_pop[i])
#                 print("nok")
#                 for i in range(self.num_pop):
#                     print(l_new_pop[i])
  

            
            print("\n\nBest Fitness: {0}\nBest Chromosome: {1} ".format(val_fitness_elite, l_val_best_chromosome))
            print("Sum of the probabiliteis in the best chromosome: {0}\n\n".format(sum(l_val_best_chromosome)))
            self.l_result.append([val_fitness_elite, sum(l_val_best_chromosome), l_val_best_chromosome])
        
        return self.l_result

In [ ]:
l_input_path_string = []
l_input_path_string.append("sample/P-Div-Var") # 0 
l_input_path_string.append("sample/P-Sched-Effect-Uni") # 1
l_input_path_string.append("sample/P-Div-Mod") # 2

l_input_path_string.append("E_DC_N3") #3
l_input_path_string.append("E_DC_N4") #4
l_input_path_string.append("E_DC_N5") #5 

l_input_path_string.append("EM_DC_N3") #6
l_input_path_string.append("EM_DC_N4") #7
l_input_path_string.append("EM_DC_N5") #8

l_input_path_string.append("I_DC_N3") #9
l_input_path_string.append("I_DC_N4") #10
l_input_path_string.append("I_DC_N5") #11

l_input_path_string.append("IM_DC_N3") #12
l_input_path_string.append("IM_DC_N4") #13
l_input_path_string.append("IM_DC_N5") #14



idx_input_path = 3

str_input = func_input_file (l_input_path_string[idx_input_path])
print("\n--> importing input file completed.")

num_secrets, num_traces, l_prob_trace, l_name_trace, l_freq_trace = func_split_input_string (str_input)
print("\n--> splitting input file completed.\n")

# func_print_input_file(num_secrets, num_traces, l_name_trace, l_freq_trace, l_prob_trace)

o_ga = GA (num_pop = 1000, num_secrets = num_secrets, num_bit = 10, num_traces = num_traces, \
           l_prob_trace = l_prob_trace, l_name_trace = l_name_trace, l_freq_trace = l_freq_trace, \
           val_mutation_prob = 0.1, val_mating_prob = 0.8, val_max_iter = 1000)

print("--> starting genetic algorithm ...\n")

l_result = o_ga.m_start(max_num_times = 3)

func_store_excel(l_input_path_string[idx_input_path], l_result)


Case_Studies/E_DC_N3

--> importing input file completed.
num_secrets
3

num_traces
31

l_name_trace
['[0-0-0, 0-0-0, 0-0-0, 0-0-1, 0-0-1, 0-1-1, 0-1-1]', '[0-0-0, 0-0-0, 0-0-0, 0-0-0, 0-0-0, 1-0-0, 1-0-1]', '[0-0-0, 0-0-0, 0-0-0, 0-0-0, 0-0-0, 1-0-0, 1-1-0]', '[0-0-0, 0-0-0, 0-0-0, 0-0-0, 0-0-0, 0-0-1, 0-1-1]', '[0-0-0, 0-0-0, 0-0-0, 0-0-0, 0-0-1, 0-0-1, 0-1-1]', '[0-0-0, 0-0-0, 0-0-0, 0-0-0, 0-0-0, 0-1-0, 0-1-1]', '[0-0-0, 0-0-0, 0-0-0, 0-0-0, 1-0-0, 1-0-1, 1-0-1]', '[0-0-0, 0-0-0, 0-0-0, 0-0-0, 1-0-0, 1-1-0, 1-1-0]', '[0-0-0, 0-0-0, 0-0-0, 1-0-0, 1-0-0, 1-0-1, 1-0-1]', '[0-0-0, 0-0-0, 0-0-0, 1-0-0, 1-0-0, 1-1-0, 1-1-0]', '[0-0-0, 0-0-0, 0-0-0, 0-0-1, 0-0-1, 1-0-1, 1-0-1]', '[0-0-0, 0-0-0, 0-0-0, 0-1-0, 0-1-0, 0-1-1, 0-1-1]', '[0-0-0, 0-0-0, 0-0-0, 0-1-0, 0-1-0, 1-1-0, 1-1-0]', '[0-0-0, 0-0-0, 0-0-0, 0-0-0, 0-1-0, 1-1-0, 1-1-0]', '[0-0-0, 0-0-0, 0-0-0, 0-1-0, 0-1-0, 0-1-0, 1-1-0]', '[0-0-0, 0-0-0, 0-0-0, 0-0-0, 0-1-0, 0-1-1, 0-1-1]', '[0-0-0, 0-0-0, 0-0-0, 0-0-1, 0-0-1, 0-0-1, 1-0-1]